In [3]:
from config import config
config = config.Configuration()

In [5]:
# run previous notebooks if necessary
if not "run_prev_notebooks" in locals(): # use case: this notebook
    run_prev_notebooks = False
    %run "1_data_preparation.ipynb"
    %run "2_model.ipynb"

In [6]:
import os
import time
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
timestr = time.strftime("%Y%m%d-%H%M")

# Define callbacks for the early stopping of training, LearningRateScheduler and model checkpointing
checkpoint = ModelCheckpoint(config.model_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = False)

#reduceonplatea; It can be useful when using adam as optimizer
#Reduce learning rate when a metric has stopped improving (after some patience,reduce by a factor of 0.33, new_lr = lr * factor).
#cooldown: number of epochs to wait before resuming normal operation after lr has been reduced.
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.33,
                                   patience=4, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=4, min_lr=1e-16)

#early = EarlyStopping(monitor="val_loss", mode="min", verbose=2, patience=15)

log_dir = os.path.join(f'./logs/UNet_{timestr}_{OPTIMIZER_NAME}_{LOSS_NAME}_{config.input_shape[0]}')
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

callbacks_list = [checkpoint, tensorboard] #reduceLROnPlat is not required with adaDelta

In [ ]:
%%time
loss_history = model.fit(
        x=np.array(X_train), 
        y=np.array(y_train),
        epochs=config.NB_EPOCHS,
        steps_per_epoch=len(X_train) // config.BATCH_SIZE,
        validation_data=(X_val, y_val),
        shuffle=True,
        validation_steps=len(X_train[0]) // config.BATCH_SIZE,
        batch_size=config.BATCH_SIZE,
        callbacks=callbacks_list
        # use_multiprocessing=True
)
print("Training completed.")

In [ ]:
if not os.path.exists(config.model_path):
    os.makedirs(config.model_path)

acc     = round(100*loss_history.history['accuracy'][-1], 4)
val_acc = round(100*loss_history.history['val_accuracy'][-1], 4)
timestr = time.strftime("%Y%m%d-%H%M")
model_path = os.path.join(f'{config.model_path}trees_{timestr}_{OPTIMIZER_NAME}_{LOSS_NAME}_{config.input_shape[0]}.h5')
print(model_path)
model.save(model_path)

# The weights without the model architecture can also be saved. Just saving the weights is more efficent.
# weight_path="./saved_weights/UNet/{}/".format(timestr)
# if not os.path.exists(weight_path):
#     os.makedirs(weight_path)
# weight_path=weight_path + "{}_weights.best.hdf5".format('UNet_model')
# print(weight_path)

/home/jovyan/work/saved_data/south_africa_tree_stock/saved_models/UNet_20220314-2341_AdaDelta_weightmap_tversky_256_10.4729_0.0.h5
